In [98]:
import requests
import pandas as pd
import numpy as np
import ast

from datetime import datetime, timedelta
pd.options.display.max_columns = None

Usando el csv de "attacks_limpieza_completa y el csv con los datos climáticos del anterior pair, deberemos juntarlos en una única fuente de información.



In [99]:
df_tiburones = pd.read_csv("datos/attacks_limpieza_completa.csv", index_col=0)

df_clima = pd.read_csv("datos/tiempo_paises_1.csv", index_col=0)

In [100]:
df_clima.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,3,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",4,15,10,1015,none,0,0,235,2,USA
1,6,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",4,15,8,1014,none,0,0,180,3,USA


In [101]:
df_tiburones.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860


El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. 
Para ello:
- Cargaremos los dos ficheros de datos
- Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer: (USA, Australia, New Zealand, South Africa, Papua New Guinea)
- Del dataframe de los datos climáticos seleccionaremos todas las columnas.
- Cuando ya tengamos todos los datos deseados juntaremos los dos csv.
- Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.
- Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. 
Si intentamos hacer la media de eso no nos dará un valor real. A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

In [102]:
paises = ["usa", "australia", "new zealand", "south africa", "papua new guinea"]

In [103]:
df_tiburones_country =df_tiburones[df_tiburones["country"].isin(paises)]
df_tiburones_country["country"].unique()

array(['usa', 'australia', 'south africa', 'new zealand',
       'papua new guinea'], dtype=object)

In [104]:
# Como los datos de las columnas, respecto a los paises, en un df está en capitalize y en el otro en minúsculas vamos a unificarlo un poco.

df_clima["country"] = df_clima["country"].str.lower()

In [105]:
df_clima.head(1)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,3,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",4,15,10,1015,none,0,0,235,2,usa


In [106]:
df_tiburones_country.head(1)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327


Procedemos a limpiar las columnas del df_clima de "rh_profile" y "wind_profile" 

In [107]:
df_clima['rh_profile']= df_clima['rh_profile'].apply(ast.literal_eval)

In [108]:
df_rh= df_clima["rh_profile"].apply(pd.Series)
df_rh.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'rh': -9999}","{'layer': '900mb', 'rh': 3}","{'layer': '850mb', 'rh': 2}","{'layer': '800mb', 'rh': 2}","{'layer': '750mb', 'rh': 3}","{'layer': '700mb', 'rh': 5}","{'layer': '650mb', 'rh': 4}","{'layer': '600mb', 'rh': 2}","{'layer': '550mb', 'rh': 11}","{'layer': '500mb', 'rh': 14}","{'layer': '450mb', 'rh': 9}","{'layer': '400mb', 'rh': 6}","{'layer': '350mb', 'rh': 8}","{'layer': '300mb', 'rh': 13}","{'layer': '250mb', 'rh': 11}","{'layer': '200mb', 'rh': 3}"


In [114]:
for i in range(len(df_rh.columns)): 
    #print(df_rh_copi[0].apply(pd.Series))
    
    nombre = "rh_" + str(df_rh[i].apply(pd.Series)["layer"][0]) 
    print(nombre)
    valores = list(df_rh[i].apply(pd.Series)["rh"] )
    print(valores)
    df_clima.insert(i, nombre, valores)
    


rh_0    950mb
0    950mb
0    950mb
0    950mb
0    950mb
Name: layer, dtype: object
[-9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, 7, 5, 4, 5, 5, 6, 6, 6, 4, 2, 5, 3, 4, 5, 7, 6, 3, 1, 3, 3, 4, 5, 5, 4, 1, 0, 0, 1, 2, 3, 4, 5, 2, 0, 0, 4, 4, 5, 6, 5, 3, 1, 3, 2, 2, 4, 6, 5, 2, 1, 3, 4, 4, 5, 5, 3, 2, 2, 2, 2, 2, 2, 1, 1, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -

ValueError: cannot insert rh_0    950mb
0    950mb
0    950mb
0    950mb
0    950mb
Name: layer, dtype: object, already exists

In [113]:
df_clima.head(1)

,"rh_0 950mb\n0 950mb\n0 950mb\n0 950mb\n0 950mb\nName: layer, dtype: object","rh_0 900mb\n0 900mb\n0 900mb\n0 900mb\n0 900mb\nName: layer, dtype: object","rh_0 850mb\n0 850mb\n0 850mb\n0 850mb\n0 850mb\nName: layer, dtype: object","rh_0 800mb\n0 800mb\n0 800mb\n0 800mb\n0 800mb\nName: layer, dtype: object","rh_0 750mb\n0 750mb\n0 750mb\n0 750mb\n0 750mb\nName: layer, dtype: object","rh_0 700mb\n0 700mb\n0 700mb\n0 700mb\n0 700mb\nName: layer, dtype: object","rh_0 650mb\n0 650mb\n0 650mb\n0 650mb\n0 650mb\nName: layer, dtype: object","rh_0 600mb\n0 600mb\n0 600mb\n0 600mb\n0 600mb\nName: layer, dtype: object","rh_0 550mb\n0 550mb\n0 550mb\n0 550mb\n0 550mb\nName: layer, dtype: object","rh_0 500mb\n0 500mb\n0 500mb\n0 500mb\n0 500mb\nName: layer, dtype: object","rh_0 450mb\n0 450mb\n0 450mb\n0 450mb\n0 450mb\nName: layer, dtype: object","rh_0 400mb\n0 400mb\n0 400mb\n0 400mb\n0 400mb\nName: layer, dtype: object","rh_0 350mb\n0 350mb\n0 350mb\n0 350mb\n0 350mb\nName: layer, dtype: object","rh_0 300mb\n0 300mb\n0 300mb\n0 300mb\n0 300mb\nName: layer, dtype: object","rh_0 250mb\n0 250mb\n0 250mb\n0 250mb\n0 250mb\nName: layer, dtype: object","rh_0 200mb\n0 200mb\n0 200mb\n0 200mb\n0 200mb\nName: layer, dtype: object",timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,-9999,3,2,2,3,5,4,2,11,14,9,6,8,13,11,3,3,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",4,15,10,1015,none,0,0,235,2,usa


In [110]:
df_clima['wind_profile']= df_clima['wind_profile'].apply(ast.literal_eval)

In [111]:
df_wind= df_clima["wind_profile"].apply(pd.Series)
df_wind.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'direction': -9999, 'speed'...","{'layer': '900mb', 'direction': 305, 'speed': 3}","{'layer': '850mb', 'direction': 295, 'speed': 3}","{'layer': '800mb', 'direction': 275, 'speed': 5}","{'layer': '750mb', 'direction': 285, 'speed': 5}","{'layer': '700mb', 'direction': 300, 'speed': 6}","{'layer': '650mb', 'direction': 285, 'speed': 6}","{'layer': '600mb', 'direction': 275, 'speed': 5}","{'layer': '550mb', 'direction': 285, 'speed': 6}","{'layer': '500mb', 'direction': 295, 'speed': 7}","{'layer': '450mb', 'direction': 295, 'speed': 7}","{'layer': '400mb', 'direction': 305, 'speed': 6}","{'layer': '350mb', 'direction': 320, 'speed': 9}","{'layer': '300mb', 'direction': 315, 'speed': 10}","{'layer': '250mb', 'direction': 315, 'speed': 12}","{'layer': '200mb', 'direction': 315, 'speed': 7}"


Procedemos a unir los 2 DataFrames en uno.

In [112]:
#df_total = df_tiburones_country.merge(df_clima, on="country")
#df_total.head(2)

Guardar los resultados obtenidos en un csv que usaremos en próximos ejercicios de pair programming.